In [2]:
import re
import pandas as pd

In [37]:
text='2015-10-18 18:08:28,662 ERROR [RMCommunicator Allocator] org.apache.hadoop.mapreduce.v2.app.rm.RMContainerAllocator: ERROR IN CONTACTING RM.'

In [38]:
h,m=re.match(r'(.*?\[.*?\]\s[\w\.]+:)(.*)',text).groups()

In [39]:
[token for token in re.split(r'(\[|\])|\s|((?<!\d\d)\:)',h) if token!='' and token!=None]

['2015-10-18',
 '18:08:28,662',
 'ERROR',
 '[',
 'RMCommunicator',
 'Allocator',
 ']',
 'org.apache.hadoop.mapreduce.v2.app.rm.RMContainerAllocator',
 ':']

In [14]:
[token for token in re.split(r'(\[|\])|\s|((?<!\d\d)\:)',m) if token!='' and token!=None]

['ERROR', 'IN', 'CONTACTING', 'RM.']

In [15]:
f=open('./hadoop_10k.log')
lines=f.readlines()

In [16]:
header=[]
msg=[]
for line in lines:
    h,m=re.match(r'(.*?\[.*?\]\s[\w\.]+:)(.*)',line).groups()
    header.append(h.strip())
    msg.append(m.strip())

In [17]:
header_tokens=[]
for h in header:
    tokens=[token for token in re.split(r'(\[|\])|\s|((?<!\d\d)\:)',h) if token!='' and token!=None]
    header_tokens.append(tokens)

In [57]:
msg_tokens=[]
for m in msg:
    tokens=[token for token in re.split(r'\[|\]|\s|((?<!\d\d)\:(?!\/\/))',m) if token!='' and token!=None]
    msg_tokens.append(tokens)

In [58]:
msg_tokens[0]

['Progress',
 'of',
 'TaskAttempt',
 'attempt_1445182159119_0004_m_000000_0',
 'is',
 ':',
 '0.54488474']

In [59]:
df=pd.DataFrame({'header':header,'msg':msg,'header_tokens':header_tokens,'msg_tokens':msg_tokens})

In [60]:
df.to_csv('./hadoop_parsed.csv')

In [61]:
header_anno=[]
for tokens in header_tokens:
    anno=[[token,'<O>'] for token in tokens]
    anno[0][1],anno[1][1],anno[2][1],anno[-2][1]='<DATE>','<TIME>','<LVL>','<CLS>'


    header_anno.append(anno)

In [62]:
def sub_tag(series, patt_tag_list):
    def _sub_tag(series, patt, tag):
        for line in series:
            for pair in line:
                if re.search(patt,pair[0]):
                    pair[1]=tag
    for patt,tag in patt_tag_list:
        _sub_tag(series,patt,tag)

In [78]:
msg_anno=[]
for tokens in msg_tokens:
    anno=[[token,'<O>'] for token in tokens]

    msg_anno.append(anno)

In [79]:
re_date=r'\d{2}\/\d{2}\/\d{4}'
re_time=r'^\d{2}:\d{2}$'

re_obj=r'[a-zA-Z]+-\d{1,3}$'
re_cls=r'(\.([A-Z]+[a-z]*)+)|([A-Z]+[a-z]+){2,}'
re_func=r'\S*\(\)$'
re_path=r'^\'?\/'

re_host=r':\d{3,5}$'
re_url=r'[a-z]+:\/\/\S*'


In [80]:
sub_tag(msg_anno,
    [(re_cls,'<CLS>'),
    (re_func,'<FUNC>'),
    (re_path,'<PATH>'),
    (re_host,'<HOST>'),
    (re_url,'<URL>')])

In [81]:
df=pd.DataFrame({
    'msg_anno':msg_anno,
    'header_anno':header_anno,
    'header':header,
    'msg':msg,
    'header_tokens':header_tokens,
    'msg_tokens':msg_tokens})
df.to_csv('./hadoop_parsed.csv')

In [140]:
for line in df.msg_anno[10:20]:
    print(str(line)+'\n')

[['Client', '<O>'], ['environment', '<O>'], [':', '<O>'], ['java.library.path', '<O>'], ['=', '<O>'], ['/usr/java/packages/lib/amd64:/usr/lib64:/lib64:/lib', '<PATH>'], [':', '<O>'], ['/usr/lib', '<PATH>'], ['(', '<O>'], ['org.apache.zookeeper.ZooKeeper', '<CLS>'], [')', '<O>']]

[['Client', '<O>'], ['environment', '<O>'], [':', '<O>'], ['java.io.tmpdir', '<O>'], ['=', '<O>'], ['/tmp', '<PATH>'], ['(', '<O>'], ['org.apache.zookeeper.ZooKeeper', '<CLS>'], [')', '<O>']]

[['Client', '<O>'], ['environment', '<O>'], [':', '<O>'], ['java.compiler', '<O>'], ['=', '<O>'], ['<NA>', '<O>'], ['(', '<O>'], ['org.apache.zookeeper.ZooKeeper', '<CLS>'], [')', '<O>']]

[['Client', '<O>'], ['environment', '<O>'], [':', '<O>'], ['os.name', '<O>'], ['=', '<O>'], ['Linux', '<O>'], ['(', '<O>'], ['org.apache.zookeeper.ZooKeeper', '<CLS>'], [')', '<O>']]

[['Client', '<O>'], ['environment', '<O>'], [':', '<O>'], ['os.arch', '<O>'], ['=', '<O>'], ['amd64', '<O>'], ['(', '<O>'], ['org.apache.zookeeper.ZooKee

In [22]:
text

'2015-10-18 18:08:28,662 ERROR [RMCommunicator Allocator] org.apache.hadoop.mapreduce.v2.app.rm.RMContainerAllocator: ERROR IN CONTACTING RM.'

In [31]:
from nltk import word_tokenize
from nltk import sent_tokenize

In [36]:
s='nova-api.log.1.2017-05-16_13:53:08 2017-05-16 00:02:55.621 25743 INFO nova.api.openstack.compute.server_external_events [req-e265ea4c-ba4b-44f0-b086-7420e34e17cb f7b8d1f1d4d44643b07fa10ca7d021fb e9746973ac574c6b8a9e8857f56a7608 - - -] Creating event network-vif-plugged:1cf11429-1563-4f6c-823c-91ba6e0f6675 for instance 95960536-049b-41f6-9049-05fc479b6a7c'
print(word_tokenize(s))

['nova-api.log.1.2017-05-16_13:53:08', '2017-05-16', '00:02:55.621', '25743', 'INFO', 'nova.api.openstack.compute.server_external_events', '[', 'req-e265ea4c-ba4b-44f0-b086-7420e34e17cb', 'f7b8d1f1d4d44643b07fa10ca7d021fb', 'e9746973ac574c6b8a9e8857f56a7608', '-', '-', '-', ']', 'Creating', 'event', 'network-vif-plugged:1cf11429-1563-4f6c-823c-91ba6e0f6675', 'for', 'instance', '95960536-049b-41f6-9049-05fc479b6a7c']
